# 🚀 Customize `gpt-oss` model using SageMaker HyperPod recipes and training jobs

---
In this notebook, we use [SageMaker HyperPod recipes](https://github.com/aws/sagemaker-hyperpod-recipes) to fine-tune the GPT-OSS models. Recipes support fine-tuning the following latest released GPT-OSS models,
* [openai/gpt-oss-20b](https://huggingface.co/openai/gpt-oss-20b)
* [openai/gpt-oss-120b](https://huggingface.co/openai/gpt-oss-120b)

In this notebook, we show you how to use the recipes with SageMaker training jobs. To run recipes on SageMaker HyperPod, see [finetune_gpt_oss_hyperpod_recipes_eks.ipynb](https://github.com/aws-samples/amazon-sagemaker-generativeai/blob/main/3_distributed_training/models/openai--gpt-oss/finetune_gpt_oss_hyperpod_recipes_eks.ipynb)

**What are GPT‑OSS Models?**

OpenAI released **gpt‑oss‑120b** and **gpt‑oss‑20b** on **August 5, 2025**—its first open‑weight language models since GPT‑2. These models are provided under the **Apache 2.0 license**, enabling both commercial and non-commercial use with full access to the model weights.

- **gpt‑oss‑120b**  
  - ~117 billion parameters, but only ~5.1 billion active per token via Mixture‑of‑Experts (MoE) routing  
  - 36 layers, 128 experts total, with 4 active per token  
  - Supports up to **128 k context length** using dense + sparse attention, grouped multi‑query attention, and RoPE

- **gpt‑oss‑20b**  
  - ~21 billion parameters, ~3.6 billion active per token  
  - 24 layers, 32 total experts, with 4 active per token  
  - Same efficient attention and context‑length capabilities as the large variant 

These models support **chain‑of‑thought (CoT) reasoning**, structured outputs, and are compatible with the OpenAI Responses API. You can adjust reasoning effort (low/medium/high) with a simple system message—balancing latency against performance.

- **gpt‑oss‑120b** matches or exceeds the performance of OpenAI’s proprietary **o4‑mini** model on benchmarks such as Codeforces (coding), MMLU and HLE (general reasoning), HealthBench (health), and AIME (competition math).
- **gpt‑oss‑20b**, despite its smaller size, outperforms **o3‑mini** across similar benchmarks, especially in mathematics and coding domains.
---

In [ ]:
%pip install -Uq sagemaker datasets==4.0.0

Make sure you're on the latest version of SageMaker Python SDK (2.251.0 or later) before proceeding.

In [ ]:
import boto3
import sagemaker
print(sagemaker.__version__)

### Set up the environment

In [ ]:
%%time
import os

import boto3
import sagemaker
from sagemaker import get_execution_role
from sagemaker.pytorch import PyTorch

role = (
    get_execution_role()
)  # or provide a pre-existing role ARN as an alternative to creating a new role
print(f"SageMaker Execution Role: {role}")

client = boto3.client("sts")
account = client.get_caller_identity()["Account"]
print(f"AWS account: {account}")

session = boto3.session.Session()
region = session.region_name
print(f"AWS region: {region}")

sm_boto_client = boto3.client("sagemaker")
sagemaker_session = sagemaker.session.Session(boto_session=session)

# get default bucket
default_bucket = sagemaker_session.default_bucket()
default_bucket_prefix = sagemaker_session.default_bucket_prefix
default_bucket_prefix_path = ""

# If a default bucket prefix is specified, append it to the s3 path
if default_bucket_prefix:
    default_bucket_prefix_path = f"/{default_bucket_prefix}"

print("Default bucket for this session: ", default_bucket)

## Data tokenization

We now preprocesses the Multilingual-Thinking dataset for fine-tuning using HyperPod recipes:
1. Load the dataset from HuggingFace's repository
2. Initialize a tokenizer for the GPT model
3. Apply chat template formatting to the messages
4. Preprocesses the data by:
    - Tokenizing the text with max length of 4096 tokens
    - Creating labels for the input sequences
    - Handling padding tokens by setting them to -100 (ignored in loss calculation)
5. Remove unnecessary columns and saves the processed dataset to disk

In [ ]:
from datasets import load_dataset

from transformers import AutoTokenizer
import numpy as np

dataset = load_dataset("HuggingFaceH4/Multilingual-Thinking", split="train")

tokenizer = AutoTokenizer.from_pretrained("openai/gpt-oss-20b")
messages = dataset[0]["messages"]
conversation = tokenizer.apply_chat_template(messages, tokenize=False)
print(conversation)

def preprocess_function(example):
    return tokenizer.apply_chat_template(example['messages'], return_dict=True, padding="max_length", max_length=4096, truncation=True)

def label(x):
    x["labels"]=np.array(x["input_ids"])
    x["labels"][x["labels"]==tokenizer.pad_token_id]=-100
    x["labels"]=x["labels"].tolist()
    return x
dataset = dataset.map(preprocess_function, remove_columns=['reasoning_language', 'developer', 'user', 'analysis', 'final','messages'])
dataset = dataset.map(label)
dataset.save_to_disk("./multilingual_4096")

Let's upload this pre-processed data to S3 for use by our training job

In [ ]:
import boto3
import os

def upload_directory(local_dir, bucket_name, s3_prefix=''):
    s3_client = boto3.client('s3')
    
    for root, dirs, files in os.walk(local_dir):
        for file in files:
            local_path = os.path.join(root, file)
            # Calculate relative path for S3
            relative_path = os.path.relpath(local_path, local_dir)
            s3_path = os.path.join(s3_prefix, relative_path).replace("\\", "/")
            
            print(f"Uploading {local_path} to {s3_path}")
            s3_client.upload_file(local_path, bucket_name, s3_path)

upload_directory('./multilingual_4096/', default_bucket, '/datasets/multilingual_4096')

## Fine-tune the model

We'll use the Pytorch estimator to spin up the training job. Use the `recipe_overrides` to override any recipe configurations. In this case, since we use SageMaker training jobs, we will update the data locations and model locations to use the `/opt/ml` locations.

Note the `hf_model_name_or_path` parameter - this lets our recipe know to use the GPT-OSS 120B model for fine-tuning. You will need a `ml.p5.48xlarge` instance to run the fine-tuning job, so make sure you have sufficient quotas set through Service quotas.

In [ ]:
import os
import sagemaker,boto3

from sagemaker.pytorch import PyTorch

sagemaker_session = sagemaker.Session()
role = sagemaker.get_execution_role()

bucket = sagemaker_session.default_bucket()
output = os.path.join(f"s3://{bucket}", "output")

recipe_overrides = {
    "run": {
        "results_dir": "/opt/ml/model",
    },
    "exp_manager": {
        "exp_dir": "",
        "explicit_log_dir": "/opt/ml/output/tensorboard",
        "checkpoint_dir": "/opt/ml/checkpoints",
    },
    "model": {
        "data": {
            "train_dir": "/opt/ml/input/data/train",
            "val_dir": "/opt/ml/input/data/val",
        },
        "hf_model_name_or_path": "openai/gpt-oss-120b",
    },
    "use_smp_model": "false",
}

estimator = PyTorch(
  output_path=output,
  base_job_name=f"gpt-oss-recipe",
  role=role,
  instance_type="ml.p5.48xlarge",
  training_recipe="fine-tuning/gpt_oss/hf_gpt_oss_120b_seq4k_gpu_lora",
  recipe_overrides=recipe_overrides,
  sagemaker_session=sagemaker_session,
  image_uri="658645717510.dkr.ecr.us-west-2.amazonaws.com/smdistributed-modelparallel:sm-pytorch_gpt_oss_patch_pt-2.7_cuda12.8",
)

estimator.fit(inputs={"train": f"s3://{bucket}/datasets/multilingual_4096/", "val": f"s3://{bucket}/datasets/multilingual_4096/"}, wait=True)


In [ ]:
s3_model_data_uri = estimator.model_data

That's it, you have fine-tuned the GPT-OSS 120B model on your custom data. To deploy the model for inference, follow the steps in [finetune_gpt_oss.ipynb](https://github.com/aws-samples/amazon-sagemaker-generativeai/blob/main/3_distributed_training/models/openai--gpt-oss/finetune_gpt_oss.ipynb).